In [1]:
import os
import pymongo
from pymongo import MongoClient
from collections import Counter
import pickle as pk
import pandas as pd
from scipy import spatial
from sklearn.preprocessing import normalize
os.chdir("..")
os.chdir("..")

In [2]:
art_client = MongoClient('mongodb://10.237.26.154:27017/')
art_db = art_client['media-db']

In [6]:
x = art_db.articles.find({})

## Location to article mapping

In [7]:
city_map={}
state_map={}
i=0
for art in x:
    i+=1
    try:
        ent=art['entities']
    except:
        continue
    for en in ent:
        if(en['type']=="City"):
            if en['name'].lower() in city_map.keys():
                city_map[en['name'].lower()].append(art['_id'])
            else:
                city_map[en['name'].lower()] = []
                city_map[en['name'].lower()].append(art['_id'])
        if(en['type']=="ProvinceOrState"):
            if en['name'].lower() in state_map.keys():
                state_map[en['name'].lower()].append(art['_id'])
            else:
                state_map[en['name'].lower()] = []
                state_map[en['name'].lower()].append(art['_id'])
    if(i==100):
        break

In [8]:
file1 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/city_map.pk",'wb')
file2 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/state_map.pk",'wb')
pk.dump(city_map,file1)
pk.dump(state_map,file2)
file1.close()
file2.close()

In [13]:
file1 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/city_map.pk",'rb')
file2 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/state_map.pk",'rb')
city_map = pk.load(file1)
state_map = pk.load(file2)
file1.close()
file2.close()

## Extracting news articles

In [125]:
date = ["2017-01-","2017-02-"]
district = "rudraprayag"
state = "punjab"

In [12]:
text = []
if district in city_map.keys():
    for it in city_map[district]:
        x = art_db.articles.find({'_id': it})
        for temp in x:
            for d in date:
                if d in temp['publishedDate']:
                    text.append(temp['text'])

NameError: name 'district' is not defined

In [ ]:
len(text)

## Extract top keywords from text (Start from here)

In [2]:
pwd = os.getcwd( )
file1 = open(pwd+"/Mass Media/Data/city_map_big.pk",'rb')
file2 = open(pwd+"/Mass Media/Data/state_map_big.pk",'rb')
city_map = pk.load(file1)
state_map = pk.load(file2)
file1.close()
file2.close()

In [3]:
state = "up"
districts = open(pwd+"/Mass Media/Data/Districts/"+state+".txt","r").read().split('\n')
districts = [x.lower() for x in districts]
art_thresh=1000
good = []
date = [""]
queryf = 0
for district in districts:
    query = district
    one = 0
    try:
        one = len(city_map[query])
    except Exception:
        print("city",query,0)
    try:
        one+=len(state_map[query])
    except Exception:
        print("state",query,0)
    print(query,one)
    if one>=art_thresh:
        good.append(query)
good

agra 11437
aligarh 892
allahabad 22225
ambedkar nagar 125
amethi 252
amroha 119
auraiya 226
azamgarh 841
bagpat 17
bahraich 327
ballia 554
balrampur 194
state banda 0
banda 242
barabanki 466
bareilly 986
basti 199
bijnor 270
budaun 37
bulandshahr 452
chandauli 566
chitrakoot 301
deoria 294
etah 325
etawah 670
faizabad 795
farrukhabad 191
fatehpur 425
firozabad 304
gautam buddh nagar 87
ghaziabad 2408
ghazipur 800
gonda 272
gorakhpur 1399
hamirpur 363
hapur 231
hardoi 322
hathras 185
jalaun 157
jaunpur 730
jhansi 744
kannauj 324
kanpur dehat 395
kanpur nagar 52
kasganj 89
kaushambi 326
kushinagar 227
lakhimpur kheri 164
lalitpur 144
lucknow 65179
maharajganj 167
mahoba 194
mainpuri 342
mathura 1134
mau 337
meerut 7931
mirzapur 758
moradabad 761
muzaffarnagar 816
pilibhit 193
pratapgarh 666
raebareli 97
rampur 295
saharanpur 801
sambhal 104
sant kabir nagar 80
bhadohi 254
shahjahanpur 233
shamli 440
shravasti 82
siddharthnagar 46
sitapur 388
sonbhadra 252
sultanpur 364
unnao 678
varanasi

['agra',
 'allahabad',
 'ghaziabad',
 'gorakhpur',
 'lucknow',
 'mathura',
 'meerut',
 'varanasi']

In [79]:
ind=7
query = good[ind]
query

'varanasi'

In [5]:
from rake_nltk import rake
import json, nltk, math, os
# nltk.download('wordnet')
import numpy as np
# from ExtractSentences import ExtractSentences
from pymongo import MongoClient
from collections import Counter, defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from stanfordcorenlp import StanfordCoreNLP

In [12]:
# res_folder = './OUTPUT'
# fix_keyword = "event_detection"
# os.system('mkdir -p ' + res_folder.lstrip('./'))
# ofile2 = open(res_folder + '/' + str(fix_keyword) + '_keywords.txt', 'w')  # it will contain top words measure by rake's relevance of each article
# ofile3 = open(res_folder + '/' + str(fix_keyword) + '_top100_words', 'w')  # it will contain top 100 frequent words in all articles
# ofile4 = open(res_folder + '/' + str(fix_keyword) + 'Lemmatised_articles.txt', 'w')

In [6]:
client = MongoClient('mongodb://act4dgem.cse.iitd.ac.in:27017/')
art_db = client['media-db']
temp = pd.Series.from_csv(pwd+"/Mass Media/Data/SmartStoplist.txt",sep=' ',index_col=1)
temp = temp.tolist()[1:]
rake_object = rake.Rake(temp,[],max_length=4)  # min no. of chars=3, max no. of words=5, min frequency=1
wordnet_lemmatizer = WordNetLemmatizer()

/usr/local/lib/python3.7/site-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [80]:
arr = []
city_to_month = {}
vocab = set()
ent_list = {}
removed_count = 0
selected_count = 0
count = 0
first = 0
second = 0
wc = 0
i=0
map1 = (state_map if queryf==1 else city_map)
if query in map1.keys():
    for it in map1[query]:
        i+=1
        cursor = art_db.articles.find({'_id': it})
        for art in cursor:
            for d in date:
                if d in art['publishedDate']:
                    wc=0
                    first=0
                    second=0
                    temp_date = art['publishedDate'][:7]
                    text = art['text']
                    count = count + 1
                    print(count, ": total_count")
                    try:
                        sentences = sent_tokenize(text)
                    except:
                        continue
                    new_text = ''
                    for sent in sentences:
                        line = ''
                        for word in word_tokenize(sent):
                            tword = wordnet_lemmatizer.lemmatize(word)
                            tword=tword.lower()
                            if (wc==0):
                                wc=1
                                if (tword==query):
                                    first=1
                            else:
                                if (tword==query):
                                    second=1
                            if len(tword) > 2:
                                tword = tword.lower()
                                if(tword.isalpha()):
                                    vocab.add(tword)
                                line = line + ' ' + tword
                        new_text = new_text + line + ' . '
                    if(first==1 and second==0):
                        removed_count+=1
                        continue
                    arr.append((new_text).lower())
                    if temp_date in city_to_month:
                        city_to_month[temp_date].append((new_text).lower())
                    else:
                        city_to_month[temp_date] = [(new_text).lower()]
                    selected_count = selected_count + 1
                    print(selected_count, ": selected_count")
                    try:
                        if temp_date in ent_list:
                            ent_list[temp_date].append(art['entities'])
                        else:
                            ent_list[temp_date] = [art['entities']]
                    except:
                        continue
ent_map = {}
i=0
for key in sorted(ent_list.keys()):
    ent_map[i] = ent_list[key]
    i+=1
print('Total articles: ', selected_count)
print('#Removed articles: ', removed_count)

1 : total_count
1 : selected_count
2 : total_count
2 : selected_count
3 : total_count
3 : selected_count
4 : total_count
4 : selected_count
5 : total_count
5 : selected_count
6 : total_count
6 : selected_count
7 : total_count
7 : selected_count
8 : total_count
8 : selected_count
9 : total_count
9 : selected_count
10 : total_count
10 : selected_count
11 : total_count
11 : selected_count
12 : total_count
12 : selected_count
13 : total_count
13 : selected_count
14 : total_count
14 : selected_count
15 : total_count
15 : selected_count
16 : total_count
16 : selected_count
17 : total_count
17 : selected_count
18 : total_count
18 : selected_count
19 : total_count
19 : selected_count
20 : total_count
20 : selected_count
21 : total_count
21 : selected_count
22 : total_count
22 : selected_count
23 : total_count
23 : selected_count
24 : total_count
24 : selected_count
25 : total_count
25 : selected_count
26 : total_count
26 : selected_count
27 : total_count
27 : selected_count
28 : total_count
28

234 : total_count
217 : selected_count
235 : total_count
218 : selected_count
236 : total_count
219 : selected_count
237 : total_count
220 : selected_count
238 : total_count
221 : selected_count
239 : total_count
222 : selected_count
240 : total_count
223 : selected_count
241 : total_count
224 : selected_count
242 : total_count
225 : selected_count
243 : total_count
226 : selected_count
244 : total_count
227 : selected_count
245 : total_count
228 : selected_count
246 : total_count
229 : selected_count
247 : total_count
230 : selected_count
248 : total_count
231 : selected_count
249 : total_count
232 : selected_count
250 : total_count
233 : selected_count
251 : total_count
234 : selected_count
252 : total_count
235 : selected_count
253 : total_count
236 : selected_count
254 : total_count
237 : selected_count
255 : total_count
238 : selected_count
256 : total_count
239 : selected_count
257 : total_count
258 : total_count
240 : selected_count
259 : total_count
260 : total_count
261 : tota

485 : total_count
398 : selected_count
486 : total_count
487 : total_count
399 : selected_count
488 : total_count
489 : total_count
400 : selected_count
490 : total_count
401 : selected_count
491 : total_count
402 : selected_count
492 : total_count
403 : selected_count
493 : total_count
404 : selected_count
494 : total_count
495 : total_count
405 : selected_count
496 : total_count
406 : selected_count
497 : total_count
407 : selected_count
498 : total_count
499 : total_count
500 : total_count
408 : selected_count
501 : total_count
502 : total_count
503 : total_count
409 : selected_count
504 : total_count
505 : total_count
410 : selected_count
506 : total_count
411 : selected_count
507 : total_count
412 : selected_count
508 : total_count
413 : selected_count
509 : total_count
414 : selected_count
510 : total_count
415 : selected_count
511 : total_count
416 : selected_count
512 : total_count
417 : selected_count
513 : total_count
418 : selected_count
514 : total_count
419 : selected_coun

748 : total_count
572 : selected_count
749 : total_count
750 : total_count
573 : selected_count
751 : total_count
752 : total_count
574 : selected_count
753 : total_count
754 : total_count
575 : selected_count
755 : total_count
756 : total_count
757 : total_count
758 : total_count
759 : total_count
576 : selected_count
760 : total_count
761 : total_count
577 : selected_count
762 : total_count
763 : total_count
578 : selected_count
764 : total_count
765 : total_count
766 : total_count
767 : total_count
768 : total_count
579 : selected_count
769 : total_count
770 : total_count
771 : total_count
772 : total_count
773 : total_count
580 : selected_count
774 : total_count
581 : selected_count
775 : total_count
776 : total_count
582 : selected_count
777 : total_count
583 : selected_count
778 : total_count
779 : total_count
780 : total_count
781 : total_count
782 : total_count
783 : total_count
784 : total_count
785 : total_count
786 : total_count
584 : selected_count
787 : total_count
788 : t

1026 : total_count
728 : selected_count
1027 : total_count
1028 : total_count
1029 : total_count
729 : selected_count
1030 : total_count
730 : selected_count
1031 : total_count
731 : selected_count
1032 : total_count
732 : selected_count
1033 : total_count
733 : selected_count
1034 : total_count
734 : selected_count
1035 : total_count
1036 : total_count
1037 : total_count
735 : selected_count
1038 : total_count
736 : selected_count
1039 : total_count
1040 : total_count
737 : selected_count
1041 : total_count
1042 : total_count
738 : selected_count
1043 : total_count
739 : selected_count
1044 : total_count
740 : selected_count
1045 : total_count
741 : selected_count
1046 : total_count
742 : selected_count
1047 : total_count
1048 : total_count
1049 : total_count
743 : selected_count
1050 : total_count
1051 : total_count
1052 : total_count
1053 : total_count
744 : selected_count
1054 : total_count
745 : selected_count
1055 : total_count
1056 : total_count
746 : selected_count
1057 : total

...

19947 : total_count
14246 : selected_count
19948 : total_count
14247 : selected_count
19949 : total_count
14248 : selected_count
19950 : total_count
14249 : selected_count
19951 : total_count
14250 : selected_count
19952 : total_count
14251 : selected_count
19953 : total_count
14252 : selected_count
19954 : total_count
14253 : selected_count
19955 : total_count
14254 : selected_count
19956 : total_count
14255 : selected_count
19957 : total_count
14256 : selected_count
19958 : total_count
14257 : selected_count
19959 : total_count
14258 : selected_count
19960 : total_count
14259 : selected_count
19961 : total_count
14260 : selected_count
19962 : total_count
19963 : total_count
19964 : total_count
14261 : selected_count
19965 : total_count
14262 : selected_count
19966 : total_count
19967 : total_count
14263 : selected_count
19968 : total_count
19969 : total_count
14264 : selected_count
19970 : total_count
14265 : selected_count
19971 : total_count
14266 : selected_count
19972 : total_cou

14439 : selected_count
20155 : total_count
14440 : selected_count
20156 : total_count
14441 : selected_count
20157 : total_count
20158 : total_count
14442 : selected_count
20159 : total_count
14443 : selected_count
20160 : total_count
14444 : selected_count
20161 : total_count
14445 : selected_count
20162 : total_count
14446 : selected_count
20163 : total_count
14447 : selected_count
20164 : total_count
20165 : total_count
20166 : total_count
14448 : selected_count
20167 : total_count
14449 : selected_count
20168 : total_count
14450 : selected_count
20169 : total_count
14451 : selected_count
20170 : total_count
14452 : selected_count
20171 : total_count
14453 : selected_count
20172 : total_count
14454 : selected_count
20173 : total_count
14455 : selected_count
20174 : total_count
14456 : selected_count
20175 : total_count
14457 : selected_count
20176 : total_count
14458 : selected_count
20177 : total_count
14459 : selected_count
20178 : total_count
14460 : selected_count
20179 : total_

14795 : selected_count
20567 : total_count
14796 : selected_count
20568 : total_count
14797 : selected_count
20569 : total_count
14798 : selected_count
20570 : total_count
14799 : selected_count
20571 : total_count
14800 : selected_count
20572 : total_count
14801 : selected_count
20573 : total_count
14802 : selected_count
20574 : total_count
14803 : selected_count
20575 : total_count
14804 : selected_count
20576 : total_count
14805 : selected_count
20577 : total_count
14806 : selected_count
20578 : total_count
14807 : selected_count
20579 : total_count
14808 : selected_count
20580 : total_count
14809 : selected_count
20581 : total_count
14810 : selected_count
20582 : total_count
14811 : selected_count
20583 : total_count
14812 : selected_count
20584 : total_count
14813 : selected_count
20585 : total_count
14814 : selected_count
20586 : total_count
14815 : selected_count
20587 : total_count
14816 : selected_count
20588 : total_count
14817 : selected_count
20589 : total_count
14818 : sel

20773 : total_count
14998 : selected_count
20774 : total_count
14999 : selected_count
20775 : total_count
15000 : selected_count
20776 : total_count
15001 : selected_count
20777 : total_count
15002 : selected_count
20778 : total_count
15003 : selected_count
20779 : total_count
15004 : selected_count
20780 : total_count
15005 : selected_count
20781 : total_count
15006 : selected_count
20782 : total_count
15007 : selected_count
20783 : total_count
15008 : selected_count
20784 : total_count
15009 : selected_count
20785 : total_count
15010 : selected_count
20786 : total_count
15011 : selected_count
20787 : total_count
15012 : selected_count
20788 : total_count
15013 : selected_count
20789 : total_count
15014 : selected_count
20790 : total_count
15015 : selected_count
20791 : total_count
15016 : selected_count
20792 : total_count
15017 : selected_count
20793 : total_count
15018 : selected_count
20794 : total_count
15019 : selected_count
20795 : total_count
15020 : selected_count
20796 : tot

20989 : total_count
15178 : selected_count
20990 : total_count
15179 : selected_count
20991 : total_count
15180 : selected_count
20992 : total_count
20993 : total_count
15181 : selected_count
20994 : total_count
15182 : selected_count
20995 : total_count
15183 : selected_count
20996 : total_count
15184 : selected_count
20997 : total_count
20998 : total_count
20999 : total_count
15185 : selected_count
21000 : total_count
15186 : selected_count
21001 : total_count
15187 : selected_count
21002 : total_count
15188 : selected_count
21003 : total_count
15189 : selected_count
21004 : total_count
15190 : selected_count
21005 : total_count
15191 : selected_count
21006 : total_count
15192 : selected_count
21007 : total_count
15193 : selected_count
21008 : total_count
21009 : total_count
15194 : selected_count
21010 : total_count
21011 : total_count
21012 : total_count
21013 : total_count
21014 : total_count
21015 : total_count
21016 : total_count
21017 : total_count
21018 : total_count
21019 : t

In [81]:
# for key in sorted(city_to_month.keys()):
#     print(key, ": ", len(city_to_month1[key]))
vocab1 = sorted(vocab)
print("vocabulary length:" ,len(vocab))
print("no of documents:",len(arr))
# city_to_month

vocabulary length: 87933
no of documents: 15313


In [82]:
i=0
vocabulary={}
for x in vocab1:
    vocabulary[x]=i
    i+=1
vocabulary

{'aaa': 0,
 'aaage': 1,
 'aaaj': 2,
 'aaakrosh': 3,
 'aaam': 4,
 'aaarthi': 5,
 'aabad': 6,
 'aabid': 7,
 'aacha': 8,
 'aachar': 9,
 'aachidih': 10,
 'aachman': 11,
 'aadab': 12,
 'aadarsh': 13,
 'aadesh': 14,
 'aadhaar': 15,
 'aadhar': 16,
 'aadhe': 17,
 'aadhinath': 18,
 'aadi': 19,
 'aadil': 20,
 'aadinath': 21,
 'aadmi': 22,
 'aadmis': 23,
 'aadopur': 24,
 'aadukalam': 25,
 'aadya': 26,
 'aag': 27,
 'aagamishyanti': 28,
 'aage': 29,
 'aaghaz': 30,
 'aagman': 31,
 'aah': 32,
 'aahaar': 33,
 'aahatein': 34,
 'aahi': 35,
 'aahvahan': 36,
 'aai': 37,
 'aaib': 38,
 'aaiclas': 39,
 'aaimed': 40,
 'aaina': 41,
 'aaipl': 42,
 'aaj': 43,
 'aaja': 44,
 'aajata': 45,
 'aakar': 46,
 'aakash': 47,
 'aakashe': 48,
 'aakhon': 49,
 'aakhri': 50,
 'aakranta': 51,
 'aakriti': 52,
 'aakrosh': 53,
 'aal': 54,
 'aala': 55,
 'aalaap': 56,
 'aalame': 57,
 'aalamgiri': 58,
 'aalap': 59,
 'aalapur': 60,
 'aalayam': 61,
 'aalim': 62,
 'aaliya': 63,
 'aalmin': 64,
 'aaloo': 65,
 'aam': 66,
 'aamaadmiparty': 

## Keyword extraction from text (RAKE) + tfidf

In [51]:
import math
def getKeywords(arr,month_date):
    keywords = {}

    for ind, text in enumerate(arr):
        rake_object.extract_keywords_from_text(text)
        list_keyword = rake_object.get_ranked_phrases_with_scores()
        json_keywords = [{'text': k[1], 'relevance': k[0]} for k in list_keyword]
        relevance = float(0.0)
        new_keywords = []
        if (len(json_keywords) >= 1):
            relevance = float(20.0 / 100) * float(json_keywords[0]['relevance'])
        for item in json_keywords:
            if (item['relevance'] >= relevance):
                new_keywords.append(str(item['text']) + ',' + str(item['relevance']))
                if len(item['text'].split()) <= 2:
                    if item['text'] not in keywords:
                        keywords[item['text']] = 0.0
                    keywords[item['text']] = keywords[item['text']] + float(item['relevance'])
        
    relevant_words = set()
    map_to_rake = {}
    i=0
    for key,val in keywords.items():
        word = key.split()
        relevant_words.add(word[0])
        map_to_rake[word[0]] = val
        i+=1
    index_of_words={}
#     if(month_date != "NONE"):
# #     adding lda entities to keywords
#         ldakeys = set()
#         for ent in ent_map[month_date]:
#             for en in ent:
#                 ldakeys.add(en['name'].lower())
#         for key in ldakeys:
#             s = key.split()
#             for it in s:
#                 relevant_words.add(it)
            
    i=0
    for j in relevant_words:
        index_of_words[j]=i
        i+=1

    sz = len(relevant_words)
    doc_count = len(arr)
    tf = [ [0.0 for j in range(doc_count)] for i in range(sz)]
    idf = [ 0.0 for i in range(sz)]
    tf_idf = [ [0.0 for j in range(doc_count)] for i in range(sz)]
    av_tfidf = [ 0.0 for i in range(sz)]
    total_occurence = [ 0.0 for i in range(sz)]

    for i in range(len(arr)):
        s=arr[i].split()
        doc_occurence=[ 0.0 for i in range(sz)]
        for w in s:
            if w==".":
                continue
            if w in relevant_words:
                idx=index_of_words[w]
                tf[idx][i]+=1
                if doc_occurence[idx]==0:
                    doc_occurence[idx]=1
                    total_occurence[idx]+=1
        for j in range(sz):
            tf[j][i]/=len(s)
    for i in range(sz):
        idf[i]=math.log(doc_count/(1+total_occurence[i]))
    for i in range(doc_count):
        for j in range(sz):
            tf_idf[j][i]=tf[j][i]*idf[j]
            av_tfidf[j]+=tf_idf[j][i]
    final_words=[]
    i=0
    for j in relevant_words:
        final_words.append(((av_tfidf[i]**5)*map_to_rake[j],j))
#         final_words.append(((av_tfidf[i]),j))
        i+=1
    final_words.sort(key=lambda x:x[0])
    final_words.reverse()
    
    return final_words

def convert(final_words):
    global vocabulary
    vector = np.zeros((len(vocabulary),1))
    for word in final_words:
        if word[1] in vocabulary:
            vector[vocabulary[word[1]]] = word[0]
    return normalize(vector,axis=0)

## Time series data generation common

In [83]:
month_vector={}
i=0
for key in sorted(city_to_month.keys()):
    print(key)
    month_vector[key] = convert(getKeywords(city_to_month[key],i)[:30])
    i+=1

2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
2012-08
2012-09
2012-10
2012-11
2012-12
2013-01
2013-02
2013-03
2013-04
2013-05
2013-06
2013-07
2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
2018-01
2018-02
2018-03
2018-04
2018-05


## Time series data generation 1

In [ ]:
base_vector = convert(getKeywords(arr,"NONE"))
data = []
for key in sorted(city_to_month.keys()):
    data.append(1 - spatial.distance.cosine(base_vector, month_vector[key]))
len(data)

In [ ]:
file11 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/mediaData_"+query+"_1.pk",'wb')
pk.dump(data,file11)
pk.dump(city_to_month,file11)
pk.dump(base_vector,file11)
pk.dump(month_vector,file11)
file11.close()

## Time series data generation 2

In [40]:
base_vector = []
temp_arr = [[] for i in range(12)]
for key in sorted(city_to_month.keys()):
    month = int(key[5:])-1
    temp_arr[month].append(city_to_month[key])

from itertools import chain
for i in range(12):
    base_vector.append(convert(getKeywords(list(chain.from_iterable(temp_arr[i])),"NONE")))

data = []
for key in sorted(city_to_month.keys()):
    month = int(key[5:])-1
    data.append(1 - spatial.distance.cosine(base_vector[month], month_vector[key]))
len(data)

77

In [71]:
file11 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/mediaDataVectors_"+query+"_2.pk",'wb')
pk.dump(data,file11)
pk.dump(city_to_month,file11)
file11.close()

## Time series data generation 3

In [84]:
k=1
mv = []
base_vector = []
temp_arr = []

for key in sorted(city_to_month.keys()):
    month = int(key[5:])-1
    mv.append(city_to_month[key])
# print(len(mv))
for x in range(len(mv)):
    if(x>=k and x<len(mv)-k):
#         print(x)
        temp_arr.append([])
        for y in range(x-k,x+k+1):
#             print("\t",y,x-k)
            temp_arr[x-k].append(mv[y])

print("done till here")
from itertools import chain
for i in range(k):
    print(i," ",)
    base_vector.append(convert(getKeywords(list(chain.from_iterable(temp_arr[0])),i)))
for i in range(len(temp_arr)):
    print(i+k," ",)
    base_vector.append(convert(getKeywords(list(chain.from_iterable(temp_arr[i])),i+k)))
for i in range(k):
    print(k+len(temp_arr)+i," ",)
    base_vector.append(convert(getKeywords(list(chain.from_iterable(temp_arr[len(temp_arr)-1])),k+len(temp_arr)+i)))
print("done till here")
data = []
j=0
for key in sorted(city_to_month.keys()):
    data.append(1 - spatial.distance.cosine(base_vector[j], month_vector[key]))
    j+=1
len(data)

done till here
0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
10  
11  
12  
13  
14  
15  
16  
17  
18  
19  
20  
21  
22  
23  
24  
25  
26  
27  
28  
29  
30  
31  
32  
33  
34  
35  
36  
37  
38  
39  
40  
41  
42  
43  
44  
45  
46  
47  
48  
49  
50  
51  
52  
53  
54  
55  
56  
57  
58  
59  
60  
61  
62  
63  
64  
65  
66  
67  
68  
69  
70  
71  
72  
73  
74  
75  
76  
77  
78  
79  
80  
81  
82  
83  
84  
85  
86  
87  
88  
done till here


89

In [85]:
print(len(month_vector), len(base_vector))
file11 = open(pwd+"/Mass Media/Data/"+state+"/mediaDataVectors_"+query+"_big.pk",'wb')
pk.dump(data,file11)
pk.dump(month_vector,file11)
pk.dump(vocabulary,file11)
file11.close()

89 89


## temp

In [72]:
file111 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/mediaDataVectors_"+query+"_2.pk",'rb')
data = pk.load(file11)
city_to_month = pk.load(file11)
file11.close()

In [1]:
getKeywords(city_to_month["2016-10"],13)[:30]

NameError: name 'getKeywords' is not defined

In [64]:
for i in range(1,13):
    tt = str(i)
    if(i < 10):
        tt = '0'+tt
    print(tt,len(city_to_month["2015-"+tt]))

01 290
02 45
03 189
04 135
05 42
06 21
07 30
08 89
09 93
10 149
11 40
12 107


In [ ]:
# file11 = open("/Users/abhishekyadav/Documents/Study/BTP/city_to_article/citytomonth_"+query+".pk",'wb')
# pk.dump(city_to_month,file11)
# pk.dump(city_to_month,file11)
# file11.close()

## Keyword extraction from LDA (dont use now)

In [22]:
keywords = set()
for ent in ent_list:
    for en in ent:
        keywords.add(en['name'].lower())
print("##FIN")
keywords1 = keywords
keywords=set()
for key in keywords1:
    s = key.split()
    for i in s:
        keywords.add(i)
len(keywords)

##FIN


10365

In [24]:
keyy=set()
for key in sorted(city_to_month.keys()):
    print(key)
    temp = getKeywords(city_to_month[key])
    for x in temp:
        try:
            keywords.add(x[1])
            keyy.add(x[1])
        except:
            continue
print(len(keywords))
print(len(keyy))

2011-01
2011-02
2011-03
2011-04
2011-05
2011-06
2011-07
2011-08
2011-09
2011-10
2011-11
2011-12
2012-01
2012-02
2012-03
2012-04
2012-05
2012-06
2012-07
2012-08
2012-09
2012-10
2012-11
2012-12
2013-01
2013-02
2013-03
2013-04
2013-05
2013-06
2013-07
2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
17306
10595
